In [44]:
print("xxx")

xxx


In [45]:
import os
import tarfile
from six.moves import urllib

In [46]:
DownLoadRoot = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HousingPath = "datasets/housing"
HousingUrl = DownLoadRoot+HousingPath+"/housing.tgz"
#创建文件夹下载数据解压
def fetch_housing_data(housing_url,housing_path):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url,tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [47]:
import pandas as pd
housing = pd.read_csv("datasets/housing/housing.csv")
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [48]:
housing.head()
#查看具体分类
housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [49]:
#摘要（空值被忽略），std--标准差
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [50]:
#单独显示窗口
%matplotlib osx

import matplotlib.pyplot as plt
#每个属性直方图
#bins柱数，figsize图尺寸大小
housing.hist(bins=50,figsize=(20,15),)
plt.show()

In [51]:
'''
定义测试集
'''
import numpy as np
#选取测试集和训练集 （通常以2：8分割)
#每次运行结果不同
def split_test_train(data, test_radio):
    #随机排列一个序列，或者数组
    shuffled_indices = np.random.permutation(len(data))
    #获取测试集数目
    test_size = int(len(data)*test_radio)
    #将随机后的序列分别定为测试集和训练集
    test_indices = shuffled_indices[:test_size]
    train_indices = shuffled_indices[test_size:]
    #iloc通过行号来取行数据  loc通过行的具体值来取行数据
    return data.iloc[test_indices],data.iloc[train_indices]

In [52]:
#固定测试集
#1.设计固定的随机数种子 np.random.seed(42)——————在下一次获取更新的数据时都会中断,可以利用固定标识符决定是否进入测试集#%%
#2.sklearn——纯随机，无分层
from sklearn.model_selection import train_test_split
train_set,test_set =train_test_split(housing,test_size=.2,random_state=42)
#test_set,train_set = split_test_train(housing,0.2)
print('sum',len(housing),':',len(test_set),'test + ',len(train_set),'train')

sum 20640 : 4128 test +  16512 train


In [53]:
#3.创建分类（以收入分层），根据分类类型比例分层抽样
#将收入中位数除以1.5(限制收入类别的数 量)，然后使用ceil进行取整(得到离散类别)
housing['cat_income'] = np.ceil(housing['median_income']/1.5)
#最后将所有大于5的类 别合并为类别5
housing['cat_income'].where(housing['cat_income'] < 5 , 5.0, inplace=True)

from sklearn.model_selection import StratifiedShuffleSplit
#n_splits是将训练数据分成train/test对的组数
split = StratifiedShuffleSplit(n_splits=1,test_size=.2,random_state=42)
#根据cat_income对数据集分割
for train_index, test_index in split.split(housing,housing['cat_income']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
#根据收入类别的分布
print(housing['cat_income'].value_counts())
housing['cat_income'].value_counts() / len(housing)

#恢复原始数据
for set in (strat_train_set,strat_test_set):
    set.drop(["cat_income"], axis=1, inplace=True)

3.0    7236
2.0    6581
4.0    3639
5.0    2362
1.0     822
Name: cat_income, dtype: int64


In [54]:
#创建训练集副本
housing = strat_train_set.copy()
#创建散点图
#以经纬度为轴，查看密度分布
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

<AxesSubplot:xlabel='longitude', ylabel='latitude'>

In [55]:
'''
s:圆半径大小=该地人口分布
c：颜色深度=价格高低
cmap：选取预设颜色表
'''
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=.4,
             s=housing['population']/100, label='population',
             c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)


<AxesSubplot:xlabel='longitude', ylabel='latitude'>

In [56]:
#检测相关性

#1.皮尔逊相关系数
'''
在数据标准化（ [公式] ）后，Pearson相关性系数、Cosine相似度、欧式距离的平方可认为是等价的
注意：仅测量线性相关性
通常情况下通过以下取值范围判断变量的相关强度：
相关系数     0.8-1.0     极强相关
                 0.6-0.8     强相关
                 0.4-0.6     中等程度相关
                 0.2-0.4     弱相关
                 0.0-0.2     极弱相关或无相关
                 小于0        负相关
'''
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.687160
total_rooms           0.135097
housing_median_age    0.114110
households            0.064506
total_bedrooms        0.047689
population           -0.026920
longitude            -0.047432
latitude             -0.142724
Name: median_house_value, dtype: float64

In [57]:
#2.Pandas的 scatter_matrix函数
from pandas.plotting import scatter_matrix
attribute = ["median_house_value", "median_income", "total_rooms", "housing_median_age",]
#figsize:表大小
scatter_matrix(housing[attribute], figsize=(12,8))

array([[<AxesSubplot:xlabel='median_house_value', ylabel='median_house_value'>,
        <AxesSubplot:xlabel='median_income', ylabel='median_house_value'>,
        <AxesSubplot:xlabel='total_rooms', ylabel='median_house_value'>,
        <AxesSubplot:xlabel='housing_median_age', ylabel='median_house_value'>],
       [<AxesSubplot:xlabel='median_house_value', ylabel='median_income'>,
        <AxesSubplot:xlabel='median_income', ylabel='median_income'>,
        <AxesSubplot:xlabel='total_rooms', ylabel='median_income'>,
        <AxesSubplot:xlabel='housing_median_age', ylabel='median_income'>],
       [<AxesSubplot:xlabel='median_house_value', ylabel='total_rooms'>,
        <AxesSubplot:xlabel='median_income', ylabel='total_rooms'>,
        <AxesSubplot:xlabel='total_rooms', ylabel='total_rooms'>,
        <AxesSubplot:xlabel='housing_median_age', ylabel='total_rooms'>],
       [<AxesSubplot:xlabel='median_house_value', ylabel='housing_median_age'>,
        <AxesSubplot:xlabel='median_incom

In [58]:
#根据相关性找到房价与收入相关性最强
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=.1)

<AxesSubplot:xlabel='median_income', ylabel='median_house_value'>

In [59]:
#通过房间数量与其他数据组合获得其他属性寻找相关性
housing['room_per_household'] = housing['total_rooms']/housing['households']
housing['bedrooms_per_rooms'] = housing['total_bedrooms']/housing['total_rooms']
housing['population_per_household'] = housing['population']/housing['households']

corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value          1.000000
median_income               0.687160
room_per_household          0.146285
total_rooms                 0.135097
housing_median_age          0.114110
households                  0.064506
total_bedrooms              0.047689
population_per_household   -0.021985
population                 -0.026920
longitude                  -0.047432
latitude                   -0.142724
bedrooms_per_rooms         -0.259984
Name: median_house_value, dtype: float64

In [60]:
#复制数据集,提取出房价
housing = strat_train_set.drop("median_house_value", axis=1)
#标签与数据分离
housing_labels = strat_train_set["median_house_value"].copy()
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
6563,-118.13,34.20,46.0,1271.0,236.0,573.0,210.0,4.9312,INLAND
12053,-117.56,33.88,40.0,1196.0,294.0,1052.0,258.0,2.0682,INLAND
13908,-116.40,34.09,9.0,4855.0,872.0,2098.0,765.0,3.2723,INLAND
11159,-118.01,33.82,31.0,1960.0,380.0,1356.0,356.0,4.0625,<1H OCEAN


In [61]:
#解决缺失的特征
'''
1.放弃这些相应的地区
2·放弃这个属性
3·将缺失的值设置为某个值(0、平均数或者中位数等都可以)
'''
#能够找到DataFrame类型数据的空值（缺失值），将空值所在的行/列删除后，将新的DataFrame作为返回值返回
#housing.dropna(subset=["total_bedrooms"])


#housing.drop("total_bedrooms", axis=1,)
#
# median = housing["total_bedrooms"].median()
# housing["total_bedrooms"].fillna(median)
# housing

'''
4.sklearn中的imputer
'''
from sklearn.impute import SimpleImputer
#指定你要用属性的中 位数值替换该属性的缺失值
impute = SimpleImputer(strategy="median")
#中位数值只能在数值属性上计算，所以我们需要创建一个没有文本 属性的数据副本ocean_proximity
#即删除ocean_proximity
housing_num = housing.drop("ocean_proximity",axis=1)
#计算了每个属性的中位数值，并将结果存储在其实例变量statistics_中
impute.fit(housing_num)
#将计算出的中位数替换缺失的特征
#结果是一个包含转换后特征的Numpy数组
X = impute.transform(housing_num)
#放回df中
housing_tr = pd.DataFrame(X,columns=housing_num.columns)
housing_tr

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042
1,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214
2,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621
3,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839
4,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347
...,...,...,...,...,...,...,...,...
16507,-118.13,34.20,46.0,1271.0,236.0,573.0,210.0,4.9312
16508,-117.56,33.88,40.0,1196.0,294.0,1052.0,258.0,2.0682
16509,-116.40,34.09,9.0,4855.0,872.0,2098.0,765.0,3.2723
16510,-118.01,33.82,31.0,1960.0,380.0,1356.0,356.0,4.0625


In [62]:
from sklearn.preprocessing import LabelEncoder
#采用映射将文本标签转换为数值
'''
缺点：相近数值误判相似度更高
'''
encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoder = encoder.fit_transform(housing_cat)
print(encoder.classes_)
housing_cat_encoder

['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']


array([0, 0, 4, ..., 1, 0, 3])

In [63]:
from sklearn.preprocessing import OneHotEncoder
'''
给每个类别创建一个二进制的属性:当类别是“<1H OCEAN”时，一个属性为1(其他为0)，
当类别 是“INLAND”时，另一个属性为1(其他为0)，以此类推。这就是独热编码，
因为只有一个属性为1(热)，其他均为0(冷)
'''
encoder = OneHotEncoder()
#输出是一个SciPy稀疏矩阵,仅存储非零元素的位置
#reshape----转换成1列,NumPy 的reshape()函数允许一个维度等于-1，也就是“未指定”:该值由数组的长度和其余维度来推断
#fit_transform需要二维数组
housing_cat_1hot = encoder.fit_transform(housing_cat_encoder.reshape(-1,1))
# housing_cat_encoder.reshape(-1,1)
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [64]:
#使用LabelBinarizer类可以一次性完成两个转换(从文本类别转化为整数 类别，再从整数类别转换为独热向量)
#sparse_output=True给LabelBinarizer构造函数,转换为稀疏矩阵
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]])

In [65]:
#自定义转换器
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin #gives fit_transform method for free

class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

#房间数量，卧室数量，人口数量，家庭数量的索引
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
   def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
       self.add_bedrooms_per_room = add_bedrooms_per_room
   def fit(self, X, y=None):
       return self  # nothing else to do
   def transform(self, X, y=None):
       #获得家庭数量与房间、人口之间的关系
       rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
       population_per_household = X[:, population_ix] / X[:, household_ix]
       if self.add_bedrooms_per_room:
           #房间与卧室数量
           bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
           return np.c_[X, rooms_per_household, population_per_household,
                        bedrooms_per_room]
       else:
           #按行连接两个矩阵
           return np.c_[X, rooms_per_household, population_per_household]

#处理Pandas DataFrames
class DataFrameSelector(BaseEstimator, TransformerMixin):
   def __init__(self, attribute_names):
       self.attribute_names = attribute_names
   def fit(self, X, y=None):
       return self
   def transform(self, X,y=None):
       return X[self.attribute_names].values

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [66]:
#特征缩放
'''
最小-最大缩放(又叫作归一化)很简单:将值重新缩放使其最终范围 归于0到1之间。实现方法是将值减去最小值并除以最大值和最小值的差。
MinMaxScaler
标准化则完全不一样:首先减去平均值(所以标准化值的均值总是 零)，然后除以方差，从而使得结果的分布具备单位方差
StandadScaler
'''
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#流水线处理数值
#Pipeline构造函数会通过一系列名称/估算器的配对来定义步骤的序列。
# 除了最后一个是估算器之外，前面都必须是转换器(也就是说，必须有 fit_transform()方法)
num_pipeline = Pipeline([
    #1.解决特征缺失，利用中位数填充
    ('impute',SimpleImputer(strategy="median")),
    #2.转换器
    ('attrs_adder', CombinedAttributesAdder()),
    #3.特征缩放：标准化
    ('std_scaler',StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr


array([[-1.15604281,  0.77194962,  0.74333089, ..., -0.31205452,
        -0.08649871,  0.15531753],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.21768338,
        -0.03353391, -0.83628902],
       [ 1.18684903, -1.34218285,  0.18664186, ..., -0.46531516,
        -0.09240499,  0.4222004 ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.3469342 ,
        -0.03055414, -0.52177644],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.02499488,
         0.06150916, -0.30340741],
       [-1.43579109,  0.99645926,  1.85670895, ..., -0.22852947,
        -0.09586294,  0.10180567]])

In [67]:
#流水线处理分类值 LabelBinarizer
from sklearn.pipeline import FeatureUnion


#获得属性
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
'''
并行处理
每条子流水线从选择器转换器开始:只需要挑出所需的属性(数值或分 类)，
删除其余的数据，然后将生成的DataFrame转换为NumPy数组， 数据转换就完成了'''
#数值流水线
num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_attribs)),
    ('impute',SimpleImputer(strategy='median')),
    ('attrs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])
#分类值流水线
cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('label_binarizer',MyLabelBinarizer())
])
#合并
full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline',num_pipeline),
    ('cat_pipeline',cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.shape

(16512, 16)

In [68]:
#线性回归
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
#训练模型，x训练集，目标值
lin_reg.fit(housing_prepared,housing_labels)
#利用训练集的实例尝试预测
#1.获得训练集的前五个实例
some_data = housing.iloc[:5]
#2.获得真实值
some_labels = housing_labels.iloc[:5]
#3.数据处理
some_data_prepare = full_pipeline.transform(some_data)
print("predict:\t",lin_reg.predict(some_data_prepare))
print("real:\t\t",list(some_labels))

predict:	 [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
real:		 [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [69]:
#均方根差 (root mean square error，缩写 RMSE)
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
#均方误差
lin_mse = mean_squared_error(housing_labels,housing_predictions)
#平方根
lin_rmse = np.sqrt(lin_mse)
#过大，拟合不足
lin_rmse

68628.19819848923

In [70]:
#决策树
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [71]:
#随机森林
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared,housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels,housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18613.710781974863

In [72]:
#交叉验证
from sklearn.model_selection import cross_val_score
'''
执行K-折 (K-fold)交叉验证的代码:它将训练集随机分割成10个不同的子集，
每个子集称为一个折叠(fold)，然后对决策树模型进行10次训练和评估——每次挑选1个折叠进行评估，
使用另外的9个折叠进行训练。产出的结果是一个包含10次评估分数的数组'''

def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Standard deviation:",scores.std())
    print('---------')

# Scikit-Learn的交叉验证功能更倾向于使用效用函数(越大越好)而 不是成本函数(越小越好)，
# 所以计算分数的函数实际上是负的 MSE(一个负值)函数，这就是为什么先计算出-scores
# scores = cross_val_score(tree_reg,housing_prepared,housing_labels,
#                          scoring="neg_mean_squared_error",cv=10)
# tree_rmse_scores = np.sqrt(-scores)
# display_scores(tree_rmse_scores)
#
#
# scores = cross_val_score(lin_reg,housing_prepared,housing_labels,
#                          scoring="neg_mean_squared_error",cv=10)
# line_rmse_scores = np.sqrt(-scores)
# display_scores(line_rmse_scores)


scores = cross_val_score(forest_reg,housing_prepared,housing_labels,
                         scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [49563.78230128 47360.2778416  50050.33606519 52520.56879304
 49393.62643811 53843.74885311 48658.44526553 47936.56709784
 52596.66644556 50157.53025497]
Mean: 50208.154935622835
Standard deviation: 2026.0317357386982
---------


In [73]:
#保存Scikit-Learn模型
import joblib
joblib.dump(lin_reg,'models/lin_reg.model')
joblib.dump(tree_reg,'models/tree_reg.model')
joblib.dump(forest_reg,'models/forest_reg.model')
print("success")

success


In [74]:
#网格搜索
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3, 10, 30,],'max_features':[2, 4, 6, 8,]},
    {'bootstrap':[False], 'n_estimators':[3, 10,],'max_features':[2, 3, 4,]},
]
forest_reg = RandomForestRegressor()
'''
param_grid告诉Scikit-Learn，首先评估第一个dict中的n_estimator和 max_features的所有3×4=12种
超参数值组合(先不要担心这些超参数现 在意味着什么，我们将在第7章中进行解释)，
接着，尝试第二个dict中 超参数值的所有2×3=6种组合，
但这次超参数bootstrap需要设置为False
而不是True(True是该超参数的默认值)
'''
grid_search = GridSearchCV(forest_reg,param_grid,cv=5,
                           scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared,housing_labels)
#最佳参数组合
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [75]:
#最佳估算器
grid_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=30)

In [76]:
#评估分数
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

63564.13623216435 {'max_features': 2, 'n_estimators': 3}
55868.616428666515 {'max_features': 2, 'n_estimators': 10}
52910.4830145633 {'max_features': 2, 'n_estimators': 30}
60104.99982149344 {'max_features': 4, 'n_estimators': 3}
53103.737864635696 {'max_features': 4, 'n_estimators': 10}
50275.8534357944 {'max_features': 4, 'n_estimators': 30}
59477.12082303333 {'max_features': 6, 'n_estimators': 3}
52365.2390762334 {'max_features': 6, 'n_estimators': 10}
49970.62364400168 {'max_features': 6, 'n_estimators': 30}
58867.26003347439 {'max_features': 8, 'n_estimators': 3}
51977.10883488239 {'max_features': 8, 'n_estimators': 10}
50261.06651852189 {'max_features': 8, 'n_estimators': 30}
62258.99560701307 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54805.92099870599 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60085.82403352767 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
53009.07184925694 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}

In [77]:
#每个属性的相对重要程度
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ["rooms_per_hold","pop_per_hhold","bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
print(num_attribs)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances,attributes),reverse=True)


['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


[(0.32231176384508153, 'median_income'),
 (0.13711908603768433, 'INLAND'),
 (0.1037996774034098, 'pop_per_hhold'),
 (0.08540597237242688, 'longitude'),
 (0.0802697313919818, 'latitude'),
 (0.07634345955111618, 'bedrooms_per_room'),
 (0.06391991560882244, 'rooms_per_hold'),
 (0.040876251547670876, 'housing_median_age'),
 (0.017205604764985796, 'total_rooms'),
 (0.016788114329714266, 'population'),
 (0.016214187850052037, 'total_bedrooms'),
 (0.015743825259195862, 'households'),
 (0.014784995289499801, '<1H OCEAN'),
 (0.005497581309899704, 'NEAR OCEAN'),
 (0.0037065439390809554, 'NEAR BAY'),
 (1.328949937770722e-05, 'ISLAND')]

In [78]:
#评估系统
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
x_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(x_test_prepared)
final_mse = mean_squared_error(y_test,final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

47137.88488845624

In [79]:
joblib.dump(final_model,'models/final_forest_reg.model')

['models/final_forest_reg.model']